In [1]:
import os
import sys
from glob import glob
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
#!apt-get -y install libgl1-mesa-glx
import cv2
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from time import time
import torch.utils.data as data
from torchvision import transforms  
import matplotlib.pyplot as plt
import seaborn as sns
import torch.optim as optim

In [2]:
mean, std = (0.56019358 ,0.52410121, 0.501457), (0.23318603, 0.24300033 ,0.24567522)

In [3]:
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
class TestDataset(Dataset):
    def __init__(self, img_paths, transform=None):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image.data

    def __len__(self):
        return len(self.img_paths)

In [4]:
# meta 데이터와 이미지 경로를 불러옵니다.
test_dir='/opt/ml/input/data/eval'
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')
# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=mean, std=std),
    
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset
)

In [5]:
print(torch.cuda.is_available())
#  True
 
#  Returns the index of a currently selected device.
print(torch.cuda.current_device())
#  0
 
#  Returns the number of GPUs available.
print(torch.cuda.device_count())
#  1
 
#  Gets the name of a device.
print(torch.cuda.get_device_name(0))
#  'GeForce GTX 1060'
 
#  Context-manager that changes the selected device.
#  device (torch.device or int) – device index to select. 
print(torch.cuda.device(0))

True
0
1
Tesla V100-PCIE-32GB


In [ ]:
# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device=torch.device('cuda')
model_root = '/opt/ml/code/save_model/res50_stepLR_7528_089.pth'
from torchvision.models import resnet50
model = resnet50(pretrained=True)
model.fc = nn.Linear(2048,18)
model.load_state_dict(torch.load(model_root))
model.to(device)
model.eval()

In [8]:
# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for i,images in enumerate(loader):
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'res50_stepLR_7528_089.csv'), index=False)
print('test inference is done!')

test inference is done!


In [ ]:
!nvidia-smi